In [2]:
# Libraries
import tensorflow as tf
import pandas as pd
from my_data_generator import DataGenerator
from convolutional_autoencoder import ConvAutoencoder
from utils import save_reconstructed_images, create_environment, create_json
from tensorflow.keras.models import load_model
import os
import numpy as np
import cv2

In [3]:
#Hyper-parameters

input_dim = (128, 128, 1)
encoder_conv_filters = [32, 64, 64, 64]
encoder_conv_kernel_size = [3, 3, 3, 3]
encoder_conv_strides = [2, 2, 2, 2]
decoder_conv_t_filters = [64, 64, 32, 1]
decoder_conv_t_kernel_size = [3, 3, 3, 3]
decoder_conv_t_strides = [2, 2, 2, 2]
sess = tf.function()
z_dim = 200
learning_rate = 0.0005
batch_size = 8
epochs = 50
r_loss_factor = 0.4
is_training = True

In [4]:
base_folder = "/content/drive/MyDrive/convolutional_autoencoder/"
#I/O paths
run_folders = {"tsv_path": base_folder + "data/partition.csv",
               "data_path": base_folder + "data/images/",
               "model_path": base_folder + "data/Models/",
               "results_path": base_folder + "data/Results/",
               "log_filename": base_folder + "data/Results/log/CAE.log"
               }

In [5]:
# Creating the directory structure
create_environment(run_folders)

In [6]:
hyperparameters = {"input_dim": input_dim,
                   "encoder_conv_filters": encoder_conv_filters,
                   "encoder_conv_kernel_size":encoder_conv_kernel_size,
                   "encoder_conv_strides":encoder_conv_strides,
                   "decoder_conv_t_filters":decoder_conv_t_filters,
                   "decoder_conv_t_kernel_size":decoder_conv_t_kernel_size,
                   "decoder_conv_t_strides":decoder_conv_t_strides,
                   "z_dim":z_dim,
                   "learning_rate":learning_rate,
                   "batch_size":batch_size,
                   "epochs":epochs,
                   "r_loss_factor":r_loss_factor,
                   "opt":"Adam",
                   "loss_function":"mse",
                   "data_path": run_folders["data_path"]
                   }
create_json(hyperparameters, run_folders)

In [7]:
# Training phase
if is_training:
  # Data loading
  df_pneumo_2d = pd.read_csv(run_folders["tsv_path"], sep=";")
  df_pneumo_2d.columns = ['ImageID', 'Partition', 'Class']
  data_filter = df_pneumo_2d['Partition']=='train'
  # Training data loader
  data_flow_train = DataGenerator(df_pneumo_2d[data_filter],
                                  input_dim[1],
                                  input_dim[0],
                                  input_dim[2],
                                  batch_size=batch_size,
                                  path_to_img=run_folders["data_path"],
                                  shuffle = True)
  # VAlidation data loader
  data_filter = df_pneumo_2d['Partition']=='val'
  data_flow_val = DataGenerator(df_pneumo_2d[data_filter],
                                  input_dim[1],
                                  input_dim[0],
                                  input_dim[2],
                                  batch_size=batch_size,
                                  path_to_img=run_folders["data_path"],
                                  )
  # Creating the CAE
  my_CAE = ConvAutoencoder(input_dim,
                           encoder_conv_filters,
                           encoder_conv_kernel_size,
                           encoder_conv_strides,
                           decoder_conv_t_filters,
                           decoder_conv_t_kernel_size,
                           decoder_conv_t_strides,
                           z_dim)

  # Building architecture
  my_CAE.build(use_batch_norm=True, use_dropout=False, VCAE=False)
  print(my_CAE.model.summary())

  # Compiling the CAE
  my_CAE.compile(learning_rate=learning_rate, r_loss_factor=r_loss_factor)

  # Training the CAE
  steps_per_epoch = len(data_flow_train)
  H = my_CAE.train(data_flow_train, epochs, steps_per_epoch, data_flow_val, run_folders)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 128, 128, 1)]     0         
                                                                 
 encoder_conv0 (Conv2D)      (None, 64, 64, 32)        320       
                                                                 
 batch_normalization (Batch  (None, 64, 64, 32)        128       
 Normalization)                                                  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 64, 64, 32)        0         
                                                                 
 encoder_conv1 (Conv2D)      (None, 32, 32, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 32, 32, 64)        256       
 chNormalization)                                          

In [8]:
is_training = True
# Data loading
df_pneumo_2d = pd.read_csv(run_folders["tsv_path"], sep=";")
df_pneumo_2d.columns = ['ImageID', 'Partition', 'Class']
data_filter = df_pneumo_2d['Partition']=='test'
# Training data loader
data_flow_test = DataGenerator(df_pneumo_2d[data_filter],
                                input_dim[1],
                                input_dim[0],
                                input_dim[2],
                                batch_size=batch_size,
                                path_to_img=run_folders["data_path"],
                                shuffle = False)

if not is_training:
  my_CAE = ConvAutoencoder.load_model(run_folders)

print(my_CAE.model.summary())

example_batch = data_flow_test.__getitem__(index=0)
example_images = example_batch[0]

y_pred = my_CAE.model.predict(example_images)
print(y_pred.shape)

save_reconstructed_images(example_images, y_pred, run_folders)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 128, 128, 1)]     0         
                                                                 
 encoder_conv0 (Conv2D)      (None, 64, 64, 32)        320       
                                                                 
 batch_normalization (Batch  (None, 64, 64, 32)        128       
 Normalization)                                                  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 64, 64, 32)        0         
                                                                 
 encoder_conv1 (Conv2D)      (None, 32, 32, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 32, 32, 64)        256       
 chNormalization)                                          